ここでは，**再帰型ニューラルネットワーク** (Recurrent Neural Network：以下、RNN) を実装し，動かしてみましょう．

例として，姓を入力にして，どの言語かに分類し，
任意の姓を予測するタスクを解きましょう．

### data

In [ ]:
!wget data.zip https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

In [ ]:
import os
import unicodedata
import string
import torch

In [ ]:
# file確認
data_path = 'data/names/'
files = os.listdir(data_path)
print(files)

# 文字コードの変更, thanks to https://stackoverflow.com/a/518232/2809427
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# sample 
print(unicodeToAscii('Ślusàrski'))

In [ ]:
# 辞書の作成
category_lines = {}
all_categories = []

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in files:
    category = os.path.splitext(os.path.basename(data_path + filename))[0]
    all_categories.append(category)

    lines = readLines(data_path + filename)
    category_lines[category] = lines
 
n_categories = len(all_categories)

# sample
print(all_categories[0])
print(category_lines['Arabic'][0])

In [ ]:
# one-hotベクトル表現 (a-zA-Z&記号)
def letterToIndex(letter):
    return all_letters.find(letter)

# 1字 to one-hot 
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# 文字 to one-hot
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

# sample
print(letterToTensor('J'))
print(lineToTensor('Jones').size())

In [ ]:
# トレーニングデータ作成関数
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

# sample 
for i in range(5):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category: ', category, '   name: ', line)

### model  
[画像](https://torch.classcat.com/wp-content/uploads/2018/05/pytorh-intermidiate-tutorials_char_rnn_classif_network.png)を参考に下の構造のネットワークを構築しましょう．
 
```
RNN(  
  (i2h): Linear(in_features=185, out_features=128, bias=True)  
  (i2o): Linear(in_features=185, out_features=18, bias=True)  
  (softmax): LogSoftmax(dim=1)  
)  
```

In [ ]:
import torch.nn as nn
import torch.optim as optim
 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
 
        self.hidden_size = hidden_size
 
        self.i2h = nn.Linear(in_features=185, out_features=128, bias=True)  
        self.i2o = nn.Linear(in_features=185, out_features=18, bias=True)  
        self.softmax = nn.LogSoftmax(dim=1)
 
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
 
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [ ]:
hidden_size = 128
input_size = n_letters
output_size = n_categories
rnn = RNN(input_size, hidden_size, output_size)



# sample
samp_input = lineToTensor('Albert')
samp_hidden = torch.zeros(1, hidden_size)
 
samp_output, sampnext_hidden = rnn(samp_input[0], samp_hidden)
print(samp_output)

In [ ]:
## 結果を見やすくするための関数
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

#sample
print(categoryFromOutput(samp_output))

### 学習

In [ ]:
loss_function = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=0.001)

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
 
    rnn.zero_grad()
 
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)#????
 
    loss = loss_function(output, category_tensor)
    
    loss.backward()
    optimizer.step()

    return output, loss.item()

In [ ]:
print

In [ ]:
n_iters = 100000
  
losses = []
 
for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
 
    # Print iter number, loss, name and guess
    if iter % 5000 == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = 'O' if guess == category else 'X (%s)' % category
        print('Iteration: %d   Loss: %.4f   Name/estimated: %s / %s   correct:%s' % (iter, loss, line, guess, correct))
    if iter % 500 == 0:
        losses.append(loss)

In [ ]:
## 結果の確認
import matplotlib.pyplot as plt
 
plt.figure()
plt.plot(losses)

### 予測

In [ ]:
# 上位3個までの任意の姓の予測関数
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)

    with torch.no_grad():
        hidden = rnn.initHidden()
        for i in range(lineToTensor(input_line).size()[0]):
            output, hidden = rnn(lineToTensor(input_line)[i], hidden)#????

        topv, topi = output.topk(n_predictions, 1, True)
 
        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))

In [ ]:
## 任意の姓を入力しましょう
predict('michael')
predict('abcd')